# Clone repository

In [ ]:
!git clone https://github.com/1595258509/VITS_Japanese.git
%cd vits
!pip install -r requirements.txt
!sudo apt-get install espeak -y

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Unpack dataset

In [ ]:
!sudo apt-get install p7zip-full
!7z x ../drive/MyDrive/dataset.zip

# Alignment

In [ ]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

# Train

In [ ]:
!python train_ms.py -c configs/yosuga_base.json -m yosuga_base